In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
path = "./dataset.csv"

class ShotDataset(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path, index_col=0)
        self.labels = df['label']
        self.data = df.drop(columns=['label'])
        
    def expand(self, factor, band):
        xdata = [self.data]
        xlabels = [self.labels]
        samples = np.random.uniform(low=-band, high=band, size=(factor))
        for s in samples:
            xdata.append(self.data * s)
            xlabels.append(self.labels)

        self.data = pd.concat(xdata)
        self.labels = pd.concat(xlabels)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # convert to numpy and to float32 
        measurement = row.to_numpy().astype(np.float32)
        
        label = self.labels.iloc[idx]

        return measurement, label
    
ds = ShotDataset(path)
ds.expand(10, 10)

In [4]:
train_size = int(0.8 * len(ds))  #   80% for training
test_size = len(ds) - train_size
train_dataset, test_dataset = random_split(ds, [train_size, test_size])

print(f"Training set size: {len(train_dataset)}")
print(f"Testing set size: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

Training set size: 13437
Testing set size: 3360


In [5]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

In [6]:
input_size = 36
hidden_size = 64
num_layers = 2
output_size = 4

# Instantiate the model
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
import time 

start = time.perf_counter()

epochs  = 10
for epoch in range(epochs):
    for data, labels in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels.long())
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

print(f"elapsed {time.perf_counter() - start:.4f} s")

Epoch [1/10], Loss: 1.5103933811187744
Epoch [1/10], Loss: 1.5109721422195435
Epoch [1/10], Loss: 1.4740893840789795
Epoch [1/10], Loss: 1.4951674938201904
Epoch [1/10], Loss: 1.4303308725357056
Epoch [1/10], Loss: 1.468737006187439
Epoch [1/10], Loss: 1.451423168182373
Epoch [1/10], Loss: 1.44463312625885
Epoch [1/10], Loss: 1.4609580039978027
Epoch [1/10], Loss: 1.4134286642074585
Epoch [1/10], Loss: 1.4045159816741943
Epoch [1/10], Loss: 1.412936806678772
Epoch [1/10], Loss: 1.4200091361999512
Epoch [1/10], Loss: 1.3718398809432983
Epoch [1/10], Loss: 1.3954538106918335
Epoch [1/10], Loss: 1.3646315336227417
Epoch [1/10], Loss: 1.3684946298599243
Epoch [1/10], Loss: 1.2894405126571655
Epoch [1/10], Loss: 1.3295409679412842
Epoch [1/10], Loss: 1.2982063293457031
Epoch [1/10], Loss: 1.3555032014846802
Epoch [1/10], Loss: 1.270666480064392
Epoch [1/10], Loss: 1.2307531833648682
Epoch [1/10], Loss: 1.2282531261444092
Epoch [1/10], Loss: 1.2473254203796387
Epoch [1/10], Loss: 1.216119766

In [9]:
with torch.no_grad():
    total = 0
    correct = 0
    for data, labels in test_loader:
        pred = torch.argmax(model(data), dim=1)
        correct += (pred == labels).sum().item()
        total += len(labels)

    print(f"Accuracy: {correct / total} ({total})")

Accuracy: 0.993452380952381 (3360)


In [10]:
torch.save(model.state_dict(), 'shots.model')